# Tweets Spam-Ham Classifier
## Author: Luis Eduardo Ferro Diez <a href="luisedof10@gmail.com">luisedof10@gmail.com</a>, <a href='mailto:luis.ferro1@correo.icesi.edu.co'>luis.ferro1@correo.icesi.edu.co</a>

This notebook contains the steps taken to train a model to classify spam-ham tweets in Scala-Spark for later usage in the Where to Sell Products project.

## Dataset
* https://www.kaggle.com/c/twitter-spam/data

## Resources
* https://www.kaggle.com/c/twitter-spam/overview
* https://github.com/ageron/handson-ml/blob/master/03_classification.ipynb

In [1]:
spark.version

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,None,spark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res1: String = 2.4.4


In [2]:
val basePath = "/media/ohtar10/Adder-Storage/datasets/twitter/spam-ham/twitter-spam"
val training = basePath + "/train.csv"
val testing = basePath + "/test.csv"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

basePath: String = /media/ohtar10/Adder-Storage/datasets/twitter/spam-ham/twitter-spam
training: String = /media/ohtar10/Adder-Storage/datasets/twitter/spam-ham/twitter-spam/train.csv
testing: String = /media/ohtar10/Adder-Storage/datasets/twitter/spam-ham/twitter-spam/test.csv


### Load the datasets

In [11]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._

val schema = new StructType().
    add("Id", LongType, false).
    add("Tweet",StringType, false).
    add("following",DoubleType, true).
    add("followers",DoubleType, true).
    add("actions", DoubleType, true).
    add("is_retweet", DoubleType, true).
    add("location", StringType, true).
    add("Type", StringType, false)
  
val trainingDF = spark.read.
    option("header", true).
    schema(schema).
    csv(training).
    select($"Id".as("id"), 
            $"Tweet".as("tweet"), 
            $"following", 
            $"followers", 
            $"actions", 
            $"is_retweet", 
            $"location",
            when($"location".isNull, 0.0).otherwise(1.0).alias("has_location"),
            $"Type".as("type")).where("Type is not null").
    na.fill(0.0, Seq("following", "followers", "actions", "is_retweet"))

trainingDF.createOrReplaceTempView("tweets")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._
schema: org.apache.spark.sql.types.StructType = StructType(StructField(Id,LongType,false), StructField(Tweet,StringType,false), StructField(following,DoubleType,true), StructField(followers,DoubleType,true), StructField(actions,DoubleType,true), StructField(is_retweet,DoubleType,true), StructField(location,StringType,true), StructField(Type,StringType,false))
trainingDF: org.apache.spark.sql.DataFrame = [id: bigint, tweet: string ... 7 more fields]


In [12]:
%%sql
select * from tweets limit 10

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [9]:
trainingDF.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: long (nullable = true)
 |-- tweet: string (nullable = true)
 |-- following: double (nullable = false)
 |-- followers: double (nullable = false)
 |-- actions: double (nullable = false)
 |-- is_retweet: double (nullable = false)
 |-- location: string (nullable = true)
 |-- has_location: double (nullable = false)
 |-- type: string (nullable = true)



We can observe the 'Type' field corresponds to a binary class: "Spam" for spam tweets and "Quality" for genuine tweets.

We also observe that as features we can use:

* Tweet: The tweet text
* following: The amount of accounts the author of this particular tweet is following
* followers: The amount of followers the author of this particalr tweet has
* actions: The total amount of favourites, retweets and replies this particular tweet has
* is_reweet: Binary where 0 means the tweet is not a retweet and 1 that it is a retweet.
* location: The user provided location for the account.

For the non-text attributes we could train a simple classification model. However, since we are dealing with text, we need to incorporate some NLP techniques to create a feature vector out of the tweet text.

We have two options: we can either implement a stemming + wordcount algorithm, probably removing the stop words to build the feature vector. Or, we can train a LDA to discover topics among the text corpora and associate them with the target class.

Let's first create a feature vector with the non-text attributes. We will create a spark pipeline for this

### First let's train a TF based model
This is, the features from text will be taken according to the term frequency of the tweet text.

In [13]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.{StringIndexer, Tokenizer, StopWordsRemover, HashingTF, VectorAssembler}
import org.apache.spark.ml.classification.LogisticRegression

//To transform the string classes to a binary-numerical representation
val classIndexer = new StringIndexer().
    setInputCol("type").
    setOutputCol("label")
    //.setStringOrderType("alphabetDesc") //Asign 1 to Quality and 0 to Spam (Alphabetical order) NOTE: This is not supported in spark 2.2.1

//To transform the text of the tweets into tokens (array of words)    
val tokenizer = new Tokenizer().
    setInputCol("tweet").
    setOutputCol("text_tokens")

//To remove stop words like pronouns and topic marking particles from the text    
val stopWordsRemover = new StopWordsRemover().
    setInputCol("text_tokens").
    setOutputCol("text_filtered").
    setLocale("en_US") // Because most of the english tweets comes from US

//To calculate the term frequencies    
val hashingTF = new HashingTF().
    setNumFeatures(1000).
    setInputCol("text_filtered").
    setOutputCol("text_features")
    
//To assemble all the features into a single feature vector
val vectorAssembler = new VectorAssembler().
    setInputCols(Array("following", "followers", "actions", "is_retweet", "has_location", "text_features")).
    setOutputCol("features")

//To train the binary classifier for spam-vs-ham tweets    
val lr = new LogisticRegression().
    setMaxIter(100).
    setRegParam(0.001)
    
val hashingTFPipeline = new Pipeline().
    setStages(
        Array(
            classIndexer, 
            tokenizer, 
            stopWordsRemover, 
            hashingTF,
            vectorAssembler,
            lr
            ))
    
val predictions = hashingTFPipeline.fit(trainingDF).transform(trainingDF)
predictions.createOrReplaceTempView("tweets")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.{StringIndexer, Tokenizer, StopWordsRemover, HashingTF, VectorAssembler}
import org.apache.spark.ml.classification.LogisticRegression
classIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_1706446d5e54
tokenizer: org.apache.spark.ml.feature.Tokenizer = tok_7f5be71bdee6
stopWordsRemover: org.apache.spark.ml.feature.StopWordsRemover = stopWords_d625072334cb
hashingTF: org.apache.spark.ml.feature.HashingTF = hashingTF_701ec6c467d9
vectorAssembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_e92231775552
lr: org.apache.spark.ml.classification.LogisticRegression = logreg_5f3632ea871a
hashingTFPipeline: org.apache.spark.ml.Pipeline = pipeline_1b96de29838f
predictions: org.apache.spark.sql.DataFrame = [id: bigint, tweet: string ... 15 more fields]


In [17]:
%%sql
select id, tweet, following, followers, actions, is_retweet, has_location, label, prediction, probability from tweets limit 10

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [15]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

val evaluator = new BinaryClassificationEvaluator().
    setMetricName("areaUnderROC")
    
val roc = evaluator.evaluate(predictions)
println(s"Area Under ROC ${roc}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_2463086e69a8
roc: Double = 0.9902136603802713
Area Under ROC 0.9902136603802713


We got a model capable of predicting 96% of the classes in the training set. There is a chance of overfitting and a CrossValidation technique will be good to implement to ensure we are training a good model.

### Now let's train a Word2Vec based model
This is, instead of accounting for the term frequencies of the tweet's text, we will transform each tweet into a vector which has the capabilities of preserving the semantic relationships between words.

In [16]:
import org.apache.spark.ml.feature.Word2Vec

val word2Vec = new Word2Vec().
    setVectorSize(20).
    setMinCount(0).
    setInputCol("text_tokens").
    setOutputCol("text_features")
    
val word2VecPipeline = new Pipeline().
    setStages(
        Array(
            classIndexer, 
            tokenizer, 
            //stopWordsRemover, 
            word2Vec,
            vectorAssembler,
            lr
            ))
    
val predictions = word2VecPipeline.fit(trainingDF).transform(trainingDF)
predictions.createOrReplaceTempView("tweets")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.feature.Word2Vec
word2Vec: org.apache.spark.ml.feature.Word2Vec = w2v_0e4216602462
word2VecPipeline: org.apache.spark.ml.Pipeline = pipeline_466822aa3015
predictions: org.apache.spark.sql.DataFrame = [id: bigint, tweet: string ... 14 more fields]


In [18]:
%%sql
select id, tweet, following, followers, actions, is_retweet, has_location, label, prediction, probability from tweets limit 10

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [20]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

val evaluator = new BinaryClassificationEvaluator().
    setMetricName("areaUnderROC")
    
val roc = evaluator.evaluate(predictions)
println(s"Area Under ROC ${roc}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_45d5de131c5f
roc: Double = 0.959969816693628
Area Under ROC 0.959969816693628


We got 95% in the ROC curve which is slightly worse than the TF appdoach. However, we might be facing overfitting, hence probably the Word2Vec model generalizes better.

In both cases however, there is risk of overfitting, hence let’s perform a Cross Validation with the training set to ensure we are constructing good models.
Note:

These param grids are intended to find the best parameters for the model training. The caveat with the param grid is that the amount of model to train grows significantly and train this in a commodity machine can be troublesome. We executed this on EMR on a 3 node cluster, 1 master node (m4.xlarge: 4 vCPU, 16GB Ram) and 2 workers (m4.2xlarge: 8 vCPU, 32GB Ram)


In [21]:
import org.apache.spark.ml.tuning.ParamGridBuilder

val hashingTFParamGrid = new ParamGridBuilder().
    addGrid(hashingTF.numFeatures, Array(10, 25, 50)).
    addGrid(lr.regParam, Array(0.1, 0.01, 0.001)).
    addGrid(lr.maxIter, Array(10, 50, 100)).
    build()

val word2VecParamGrid = new ParamGridBuilder().
    addGrid(word2Vec.vectorSize, Array(10, 25, 50)).
    addGrid(lr.regParam, Array(0.1, 0.01, 0.001)).
    addGrid(lr.maxIter, Array(10, 50, 100)).
    build()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.tuning.ParamGridBuilder
hashingTFParamGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	logreg_5f3632ea871a-maxIter: 10,
	hashingTF_701ec6c467d9-numFeatures: 10,
	logreg_5f3632ea871a-regParam: 0.1
}, {
	logreg_5f3632ea871a-maxIter: 10,
	hashingTF_701ec6c467d9-numFeatures: 10,
	logreg_5f3632ea871a-regParam: 0.01
}, {
	logreg_5f3632ea871a-maxIter: 10,
	hashingTF_701ec6c467d9-numFeatures: 10,
	logreg_5f3632ea871a-regParam: 0.001
}, {
	logreg_5f3632ea871a-maxIter: 50,
	hashingTF_701ec6c467d9-numFeatures: 10,
	logreg_5f3632ea871a-regParam: 0.1
}, {
	logreg_5f3632ea871a-maxIter: 50,
	hashingTF_701ec6c467d9-numFeatures: 10,
	logreg_5f3632ea871a-regParam: 0.01
}, {
	logreg_5f3632ea871a-maxIter: 50,
	hashingTF_701ec6c467d9-numFeatures: 10,
	logreg_5f3632ea871a-regParam: 0.001
}, {
	logreg_5f3632ea871a-maxIter:...word2VecParamGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	logreg_5f3632ea871a-maxIter: 10,
	logreg_5f3632ea871a-regParam: 0.1,
	w2v_0e4

In [22]:
import org.apache.spark.ml.tuning.CrossValidator

//First the hashingTFPipeline
val hashingTFCV = new CrossValidator().
    setEstimator(hashingTFPipeline).
    setEvaluator(evaluator).
    setEstimatorParamMaps(hashingTFParamGrid).
    setNumFolds(5).
    setParallelism(2)

val hashingTFCModel = hashingTFCV.fit(trainingDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.tuning.CrossValidator
hashingTFCV: org.apache.spark.ml.tuning.CrossValidator = cv_682b377784ad
hashingTFCModel: org.apache.spark.ml.tuning.CrossValidatorModel = cv_682b377784ad


In [24]:
val predictions = hashingTFCModel.transform(trainingDF)
val roc = evaluator.evaluate(predictions)
println(s"Area under ROC ${roc}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.DataFrame = [id: bigint, tweet: string ... 15 more fields]
roc: Double = 0.9664722517796072
Area under ROC 0.9664722517796072


With the cross validation we got 97% with the TF based model.

Now let’s do the same for the word2vec based model.

In [25]:
//Then the word 2 vec pipeline
val word2vecCV = new CrossValidator().
    setEstimator(word2VecPipeline).
    setEvaluator(evaluator).
    setEstimatorParamMaps(word2VecParamGrid).
    setNumFolds(5).
    setParallelism(2)
  
val word2VecCVModel = word2vecCV.fit(trainingDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

word2vecCV: org.apache.spark.ml.tuning.CrossValidator = cv_ccb786b81da8
word2VecCVModel: org.apache.spark.ml.tuning.CrossValidatorModel = cv_ccb786b81da8


In [26]:
val predictions = word2VecCVModel.transform(trainingDF)
val roc = evaluator.evaluate(predictions)
println(s"Area under ROC ${roc}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.DataFrame = [id: bigint, tweet: string ... 14 more fields]
roc: Double = 0.8088541597515011
Area under ROC 0.8088541597515011


We got 80% in the evaluation, this suggests that the word2Vec model was overfitting the data, however, we might still want to use it as this is a good score.

Let’s explore the model best parameters

#### For the HashingTF model

In [29]:
import org.apache.spark.ml.PipelineModel
import org.apache.spark.ml.classification.LogisticRegressionModel

//Average metrics per param grid
val avgMetricsParamGrid = hashingTFCModel.avgMetrics

//Combibe with param grid to see how they affect the overall metrics
val combined = hashingTFParamGrid.zip(avgMetricsParamGrid)

val bestModel = hashingTFCModel.bestModel.asInstanceOf[PipelineModel]

//Explain params for each stage
val bestHashingTFNumFeatures = bestModel.stages(3).asInstanceOf[HashingTF].explainParams
val bestLRParams = bestModel.stages(5).asInstanceOf[LogisticRegressionModel].explainParams

println("#" * 10)
println(s"The best HashingTF number of features are: ")
println(bestHashingTFNumFeatures)
println("#" * 10)
println(s"The best logistic regression parameters are:")
println(bestLRParams)
println("#" * 10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.PipelineModel
import org.apache.spark.ml.classification.LogisticRegressionModel
avgMetricsParamGrid: Array[Double] = Array(0.9145250179920914, 0.9448457169665666, 0.968437156273551, 0.9144340201894247, 0.9431491259476411, 0.969148484115817, 0.9144340201894247, 0.9431491259476411, 0.969148484115817, 0.9155308768247561, 0.9461308609499272, 0.9678906547940839, 0.9154269734203883, 0.9432459497743395, 0.9689206232983162, 0.9154269734203883, 0.9432459497743395, 0.9689206232983162, 0.9166665035906965, 0.9477557882782236, 0.967594423181924, 0.9165523694940602, 0.943738096843964, 0.9688179240571554, 0.9165523694940602, 0.943738096843964, 0.9688180852104976)
combined: Array[(org.apache.spark.ml.param.ParamMap, Double)] =
Array(({
	logreg_5f3632ea871a-maxIter: 10,
	hashingTF_701ec6c467d9-numFeatures: 10,
	logreg_5f3632ea871a-regParam: 0.1
},0.9145250179920914), ({
	logreg_5f3632ea871a-maxIter: 10,
	hashingTF_701ec6c467d9-numFeatures: 10,
	logreg_5f3632ea871a-regParam: 0

The best parameters for the HashingTF suggest that the CV can be pushed further.

Now let's check the word2Vec ones.

#### For the word2vec model

In [39]:
import org.apache.spark.ml.feature.Word2VecModel

//Average metrics per param grid
val avgMetricsParamGrid = word2VecCVModel.avgMetrics

//Combibe with param grid to see how they affect the overall metrics
val combined = word2VecParamGrid.zip(avgMetricsParamGrid)

val bestModel = word2VecCVModel.bestModel.asInstanceOf[PipelineModel]

//Explain params for each stage
val bestWord2VecFeatures = bestModel.stages(2).asInstanceOf[Word2VecModel].explainParams
val bestLRParams = bestModel.stages(4).asInstanceOf[LogisticRegressionModel].explainParams

println("#" * 10)
println(s"The best Word2Vec features are: ")
println(bestWord2VecFeatures)
println("#" * 10)
println(s"The best logistic regression parameters are:")
println(bestLRParams)
println("#" * 10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.feature.Word2VecModel
avgMetricsParamGrid: Array[Double] = Array(0.9267040769817643, 0.9499469659313368, 0.9637291691154359, 0.9233550984678844, 0.9376269136248313, 0.9430189010981028, 0.9125169177937366, 0.928507508086202, 0.9314878700689965, 0.9267478314072566, 0.9493011969665435, 0.9718207935668939, 0.9248641405703333, 0.9499991035500553, 0.9725179610133754, 0.9229259391870261, 0.949949442027334, 0.9737005706383913, 0.9267478314072566, 0.9493011969665435, 0.9718208407830964, 0.9248641405703333, 0.9499973538317772, 0.9724977999705011, 0.9229259391870261, 0.9499672539914416, 0.9735091760481087)
combined: Array[(org.apache.spark.ml.param.ParamMap, Double)] =
Array(({
	logreg_5f3632ea871a-maxIter: 10,
	logreg_5f3632ea871a-regParam: 0.1,
	w2v_0e4216602462-vectorSize: 10
},0.9267040769817643), ({
	logreg_5f3632ea871a-maxIter: 10,
	logreg_5f3632ea871a-regParam: 0.01,
	w2v_0e4216602462-vectorSize: 10
},0.9499469659313368), ({
	logreg_5f3632ea871a-maxIter: 10,
	log

As for the word2vec model, we are also at the edge of the number of dimensions, which suggests that we can indeed push further to validate a better model.